In [1]:
import os
import itk                                                                
import itkwidgets
import numpy as np
import pandas as pd
import nibabel as nib
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, IntSlider, ToggleButtons

sns.set_style('darkgrid')

In [2]:
df = pd.read_csv("NFBS_Dataset_meta.csv")

In [3]:
def get_image_slider(row):
    images = []

    for file in row:
        image = nib.load(file).get_fdata()
        images.append(np.expand_dims(image, axis=3))
    
    def explore_3dimage(depth):
        plt.figure(figsize=(10, 5))
        labels = ["MRI - Skull Layers", "Skull Stripped Brain Layers", "Mask Layers"]
        for i, image in enumerate(images):
            plt.subplot(1, len(images), i+1)
            plt.imshow(image[:, :, depth, :], cmap='gray')
            plt.title(labels[i])
            plt.axis('off')

    interact(explore_3dimage, depth=(0, 191));

In [4]:
get_image_slider(df.loc[0])

interactive(children=(IntSlider(value=95, description='depth', max=191), Output()), _dom_classes=('widget-inte…

In [6]:
get_image_slider(df.loc[10])

interactive(children=(IntSlider(value=95, description='depth', max=191), Output()), _dom_classes=('widget-inte…

In [7]:
get_image_slider(df.loc[124])

interactive(children=(IntSlider(value=95, description='depth', max=191), Output()), _dom_classes=('widget-inte…